In [97]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import lit, mean, col, avg, count, monotonically_increasing_id, max, row_number
from pyspark.sql.window import Window
import os
from pandas import set_option
set_option('display.max_columns', None)
set_option('display.max_rows', None)


In [2]:
conf = SparkConf()
conf.set('spark.executor.memory', '8g')
conf.set('spark.driver.memory', '8g')
conf.set('spark.executor.cores', '4')
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
spark = SparkSession.builder.appName("eda").config(conf=conf).getOrCreate()

22/08/28 17:42:50 WARN Utils: Your hostname, gustavoalmeida resolves to a loopback address: 127.0.1.1; using 192.168.51.177 instead (on interface eth0)
22/08/28 17:42:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/28 17:42:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [34]:
arq1 = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1',
                 inferSchema='True') \
        .csv('./data/microdados2017_arq1.txt')
        
arq1.limit(10).toPandas()

,NU_ANO,CO_CURSO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO
0,2017,3,1,1,10028,5710,1,5103403,51,5
1,2017,3,1,1,10028,5710,1,5103403,51,5
2,2017,3,1,1,10028,5710,1,5103403,51,5
3,2017,3,1,1,10028,5710,1,5103403,51,5
4,2017,3,1,1,10028,5710,1,5103403,51,5
5,2017,3,1,1,10028,5710,1,5103403,51,5
6,2017,3,1,1,10028,5710,1,5103403,51,5
7,2017,3,1,1,10028,5710,1,5103403,51,5
8,2017,3,1,1,10028,5710,1,5103403,51,5
9,2017,3,1,1,10028,5710,1,5103403,51,5


In [35]:
arq2 = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1',
                 inferSchema='True') \
        .csv('./data/microdados2017_arq2.txt')
        
arq2.limit(10).toPandas()

,NU_ANO,CO_CURSO,ANO_FIM_EM,ANO_IN_GRAD,CO_TURNO_GRADUACAO
0,2017,110414,1957,2016,3
1,2017,1286531,1957,2015,4
2,2017,108616,1958,2015,3
3,2017,1341949,1958,2015,3
4,2017,9985,1959,2013,4
5,2017,1160839,1959,2013,3
6,2017,86672,1960,2006,4
7,2017,317270,1961,2013,1
8,2017,1313397,1962,2015,3
9,2017,86672,1962,2008,4


In [36]:
arq3 = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1',
                 inferSchema='True') \
        .csv('./data/microdados2017_arq3.txt')
        
arq3.limit(10).toPandas()

,NU_ANO,CO_CURSO,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,CO_RS_I1,CO_RS_I2,CO_RS_I3,CO_RS_I4,CO_RS_I5,CO_RS_I6,CO_RS_I7,CO_RS_I8,CO_RS_I9
0,2017,132,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,.,.,.,.,.,.,D,E,.
1,2017,137,8,0,0,0,27,0,5,0,CCBBCEAD,XDXAEABCXBCXDEEBBDDAAECXDCC,........,0,...........................,909000009009000000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None
2,2017,162,8,0,0,0,27,0,4,0,CCBBCEAD,XBXCXAXACCEBDBAADDBBCEEAADD,E.......,0,....B...........C..........,909090900000000000000000000,555,555,555,555,555,333,336,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,A,A,C,A,A,A,E,E,B
3,2017,162,8,0,0,0,27,0,4,0,CCBBCEAD,XBXCXAXACCEBDBAADDBBCEEAADD,........,0,...........................,909090900000000000000000000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None
4,2017,163,8,0,1,0,27,0,4,0,XCBBCEAD,EAADDAXDEBBAXXDDDBBCAEADCBX,........,90000000,...........................,900000990000000000009,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,A,A,A,C,C,A,D,E,E
5,2017,517,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None
6,2017,630,8,0,0,0,27,0,4,0,CCBBCEAD,BCBXCCACABEEDBXBCEDBCXXDEAA,........,0,...........................,900000000009000000990000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None
7,2017,630,8,0,0,0,27,0,4,0,CCBBCEAD,BCBXCCACABEEDBXBCEDBCXXDEAA,........,0,...........................,900000000009000000990000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None
8,2017,697,8,0,1,0,27,0,4,0,XCBBCEAD,EAADDAXDEBBAXXDDDBBCAEADCBX,........,90000000,...........................,900000990000000000009,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None
9,2017,750,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None


In [37]:
arq5 = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1',
                 inferSchema='True') \
        .csv('./data/microdados2017_arq5.txt')
        
arq5.limit(10).toPandas()

,NU_ANO,CO_CURSO,TP_SEXO
0,2017,132,F
1,2017,162,F
2,2017,162,F
3,2017,630,F
4,2017,750,F
5,2017,775,F
6,2017,1740,F
7,2017,1748,F
8,2017,1882,F
9,2017,1882,F


In [53]:
arq6 = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1',
                 inferSchema='True') \
        .csv('./data/microdados2017_arq6.txt')
        
arq6.limit(10).toPandas()

,NU_ANO,CO_CURSO,NU_IDADE
0,2017,102280,10
1,2017,100373,16
2,2017,101420,17
3,2017,98892,17
4,2017,99968,18
5,2017,107836,18
6,2017,86782,18
7,2017,31376,18
8,2017,70036,18
9,2017,123537,18


Qual era o número de alunos de cursos de Filosofia inscritos no ENADE 2017?

In [23]:
cursos_filosofia = set([curso[0] for curso in arq1.filter(col('CO_GRUPO').isin([3201, 3202]))\
    .select(col('CO_CURSO'))\
    .collect()])

In [24]:
cursos_filosofia

{326,
 380,
 496,
 595,
 779,
 849,
 950,
 1223,
 1321,
 1453,
 1513,
 1643,
 1738,
 1881,
 2108,
 2197,
 3168,
 4194,
 4416,
 4727,
 4866,
 5130,
 5483,
 5954,
 6417,
 7505,
 7828,
 8155,
 8338,
 8912,
 10809,
 10863,
 11229,
 11301,
 11431,
 11994,
 12321,
 12578,
 12834,
 12962,
 13097,
 13209,
 13303,
 13409,
 13577,
 13839,
 14110,
 14228,
 14346,
 14714,
 14985,
 17270,
 17346,
 17596,
 17924,
 18975,
 19895,
 20530,
 21625,
 27858,
 37274,
 40700,
 41848,
 41849,
 44362,
 45024,
 45025,
 45735,
 48044,
 49472,
 49701,
 49756,
 50042,
 51283,
 52196,
 53692,
 54780,
 58380,
 59319,
 64518,
 66597,
 66637,
 67545,
 68172,
 69002,
 71837,
 73522,
 73719,
 73972,
 74493,
 79645,
 81198,
 82605,
 83218,
 83931,
 84701,
 86672,
 86920,
 87522,
 87804,
 88504,
 88963,
 90441,
 92129,
 98144,
 99189,
 99346,
 100740,
 101292,
 101774,
 102308,
 106674,
 108342,
 108644,
 109394,
 109869,
 110414,
 113100,
 113665,
 114061,
 115686,
 116506,
 116732,
 117128,
 120737,
 121366,
 121367,
 

In [25]:
arq1.filter(col('CO_GRUPO').isin([3201, 3202]))\
    .select(col('CO_CURSO'))\
    .count()

7113

Qual é o código de UF que possui a maior média de nota geral entre os alunos dos cursos de Filosofia?

In [30]:
df_uf = arq1.select(col('CO_CURSO'), col('CO_REGIAO_CURSO'), col('CO_UF_CURSO'), col('CO_GRUPO'))\
            .distinct()


In [31]:
df_uf.limit(20).toPandas()

,CO_CURSO,CO_REGIAO_CURSO,CO_UF_CURSO,CO_GRUPO
0,635,3,35,3502
1,3846,4,42,702
2,4008,3,33,2001
3,4850,2,26,1602
4,6725,4,42,2001
5,8424,2,26,905
6,10154,3,35,1602
7,14221,4,42,5402
8,14793,3,31,5710
9,14978,4,43,21


In [33]:
arq3.join(df_uf, arq3.CO_CURSO == df_uf.CO_CURSO)\
    .filter(col('CO_GRUPO').isin([3201, 3202]))\
    .groupBy('CO_UF_CURSO')\
    .agg(avg('NT_GER').alias('media'))\
    .sort('media', ascending=False)\
    .limit(20)\
    .toPandas()


,CO_UF_CURSO,media
0,53,53.696154
1,33,50.593031
2,42,49.989100
3,31,49.866809
4,52,48.417857
5,35,48.276890
6,26,48.187324
7,50,48.172340
8,41,48.158136
9,43,47.561389


Qual é a diferença das médias de idade (arredondado para 2 dígitos decimais) entre os alunos do sexo masculino e feminino, dos cursos de filosofia?

In [112]:
df_arq5 = arq5.withColumn("id5",row_number().over(Window.partitionBy("NU_ANO").orderBy("NU_ANO")))
df_arq3 = arq3.withColumn("id3",row_number().over(Window.partitionBy("NU_ANO").orderBy("NU_ANO")))
df_arq6 = arq6.withColumn("id6",row_number().over(Window.partitionBy("NU_ANO").orderBy("NU_ANO")))

In [113]:
df_arq3.limit(10).toPandas()

,NU_ANO,CO_CURSO,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,CO_RS_I1,CO_RS_I2,CO_RS_I3,CO_RS_I4,CO_RS_I5,CO_RS_I6,CO_RS_I7,CO_RS_I8,CO_RS_I9,id3
0,2017,132,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,.,.,.,.,.,.,D,E,.,1
1,2017,137,8,0,0,0,27,0,5,0,CCBBCEAD,XDXAEABCXBCXDEEBBDDAAECXDCC,........,0,...........................,909000009009000000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,2
2,2017,162,8,0,0,0,27,0,4,0,CCBBCEAD,XBXCXAXACCEBDBAADDBBCEEAADD,E.......,0,....B...........C..........,909090900000000000000000000,555,555,555,555,555,333,336,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,A,A,C,A,A,A,E,E,B,3
3,2017,162,8,0,0,0,27,0,4,0,CCBBCEAD,XBXCXAXACCEBDBAADDBBCEEAADD,........,0,...........................,909090900000000000000000000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,4
4,2017,163,8,0,1,0,27,0,4,0,XCBBCEAD,EAADDAXDEBBAXXDDDBBCAEADCBX,........,90000000,...........................,900000990000000000009,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,A,A,A,C,C,A,D,E,E,5
5,2017,517,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,6
6,2017,630,8,0,0,0,27,0,4,0,CCBBCEAD,BCBXCCACABEEDBXBCEDBCXXDEAA,........,0,...........................,900000000009000000990000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,7
7,2017,630,8,0,0,0,27,0,4,0,CCBBCEAD,BCBXCCACABEEDBXBCEDBCXXDEAA,........,0,...........................,900000000009000000990000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,8
8,2017,697,8,0,1,0,27,0,4,0,XCBBCEAD,EAADDAXDEBBAXXDDDBBCAEADCBX,........,90000000,...........................,900000990000000000009,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,9
9,2017,750,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,10


In [114]:
df_arq5.limit(10).toPandas()

,NU_ANO,CO_CURSO,TP_SEXO,id5
0,2017,132,F,1
1,2017,162,F,2
2,2017,162,F,3
3,2017,630,F,4
4,2017,750,F,5
5,2017,775,F,6
6,2017,1740,F,7
7,2017,1748,F,8
8,2017,1882,F,9
9,2017,1882,F,10


In [115]:
df_arq6.limit(10).toPandas()

,NU_ANO,CO_CURSO,NU_IDADE,id6
0,2017,102280,10,1
1,2017,100373,16,2
2,2017,101420,17,3
3,2017,98892,17,4
4,2017,99968,18,5
5,2017,107836,18,6
6,2017,86782,18,7
7,2017,31376,18,8
8,2017,70036,18,9
9,2017,123537,18,10


In [117]:
df_arq3 = df_arq3.withColumnRenamed('CO_CURSO', 'CO_CURSO_3')
df_arq5 = df_arq5.withColumnRenamed('CO_CURSO', 'CO_CURSO_5')

In [118]:
df = df_arq3.join(df_arq5, df_arq5.id5 == df_arq3.id3)

In [119]:
df = df.join(df_arq6, df_arq6.id6 == df.id3)

In [120]:
df = df.join(df_uf, df.CO_CURSO == df_uf.CO_CURSO)

In [122]:
df.limit(20).toPandas()

,NU_ANO,CO_CURSO_3,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,CO_RS_I1,CO_RS_I2,CO_RS_I3,CO_RS_I4,CO_RS_I5,CO_RS_I6,CO_RS_I7,CO_RS_I8,CO_RS_I9,id3,NU_ANO,CO_CURSO_5,TP_SEXO,id5,NU_ANO,CO_CURSO,NU_IDADE,id6,CO_CURSO,CO_REGIAO_CURSO,CO_UF_CURSO,CO_GRUPO
0,2017,132,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,.,.,.,.,.,.,D,E,.,1,2017,132,F,1,2017,102280,10,1,102280,4,42,2001
1,2017,137,8,0,0,0,27,0,5,0,CCBBCEAD,XDXAEABCXBCXDEEBBDDAAECXDCC,........,0,...........................,909000009009000000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,2,2017,162,F,2,2017,100373,16,2,100373,2,24,702
2,2017,162,8,0,0,0,27,0,4,0,CCBBCEAD,XBXCXAXACCEBDBAADDBBCEEAADD,E.......,0,....B...........C..........,909090900000000000000000000,555,555,555,555,555,333,336,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,A,A,C,A,A,A,E,E,B,3,2017,162,F,3,2017,101420,17,3,101420,1,15,79
3,2017,162,8,0,0,0,27,0,4,0,CCBBCEAD,XBXCXAXACCEBDBAADDBBCEEAADD,........,0,...........................,909090900000000000000000000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,4,2017,630,F,4,2017,98892,17,4,98892,4,41,2001
4,2017,163,8,0,1,0,27,0,4,0,XCBBCEAD,EAADDAXDEBBAXXDDDBBCAEADCBX,........,90000000,...........................,900000990000000000009,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,A,A,A,C,C,A,D,E,E,5,2017,750,F,5,2017,99968,18,5,99968,4,42,904
5,2017,517,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,6,2017,775,F,6,2017,107836,18,6,107836,4,41,72
6,2017,630,8,0,0,0,27,0,4,0,CCBBCEAD,BCBXCCACABEEDBXBCEDBCXXDEAA,........,0,...........................,900000000009000000990000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,7,2017,1740,F,7,2017,86782,18,7,86782,3,35,72
7,2017,630,8,0,0,0,27,0,4,0,CCBBCEAD,BCBXCCACABEEDBXBCEDBCXXDEAA,........,0,...........................,900000000009000000990000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,8,2017,1748,F,8,2017,31376,18,8,31376,5,53,903
8,2017,697,8,0,1,0,27,0,4,0,XCBBCEAD,EAADDAXDEBBAXXDDDBBCAEADCBX,........,90000000,...........................,900000990000000000009,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,9,2017,1882,F,9,2017,70036,18,9,70036,3,35,79
9,2017,750,8,0,0,0,27,0,4,0,CCBBCEAD,EDXECDDCDEAXEXBAEADBCEEXEAB,........,0,...........................,9000000009090000000009000,555,333,333,333,333,333,333,333,333,333,333,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,None,None,None,None,None,None,None,None,None,10,2017,1882,F,10,2017,123537,18,10,123537,3,35,72


In [124]:
df.filter(col('CO_GRUPO').isin([3201, 3202]))\
        .groupBy('TP_SEXO')\
        .agg(mean('NU_IDADE').alias('media'))\
        .sort('media', ascending=False)\
        .limit(20)\
        .toPandas()


,TP_SEXO,media
0,M,39.283748
1,F,23.532360
2,None,NaN


Qual é a média da nota geral para alunos de filosofia, cujo curso está no estado (UF) de código 43?

In [126]:
arq3.join(df_uf, arq3.CO_CURSO == df_uf.CO_CURSO)\
    .filter(col('CO_GRUPO').isin([3201, 3202]))\
    .groupBy('CO_UF_CURSO')\
    .agg(avg('NT_GER').alias('media'))\
    .sort('media', ascending=False)\
    .filter(col('CO_UF_CURSO') == '43')\
    .limit(20)\
    .toPandas()

,CO_UF_CURSO,media
0,43,47.561389


Qual é o código do estado brasileiro que possui a maior média de idade de alunos de filosofia?

In [127]:
df.filter(col('CO_GRUPO').isin([3201, 3202]))\
        .groupBy('CO_UF_CURSO')\
        .agg(mean('NU_IDADE').alias('media'))\
        .sort('media', ascending=False)\
        .limit(20)\
        .toPandas()


,CO_UF_CURSO,media
0,32,37.992042
1,11,37.714286
2,42,37.667732
3,21,36.940092
4,15,35.815385
5,43,35.669623
6,29,34.836120
7,25,33.891026
8,33,33.691466
9,53,33.614583


Qual é o número de alunos dos cursos de Filosofia do sexo Masculino?

In [129]:
df.filter(col('TP_SEXO') == 'M')\
        .filter(col('CO_GRUPO').isin([3201, 3202])).count()


4578